# Imports

In [1]:
import os # Configure which GPU
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import or install Sionna
try:
    import sionna.phy
    import sionna.rt
except ImportError as e:
    import sys
    if 'google.colab' in sys.modules:
       # Install Sionna in Google Colab
       print("Installing Sionna and restarting the runtime. Please run the cell again.")
       os.system("pip install sionna")
       os.kill(os.getpid(), 5)
    else:
       raise e

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

import numpy as np

# For link-level simulations
from sionna.phy.channel import OFDMChannel, CIRDataset
from sionna.phy.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.phy.utils import ebnodb2no, PlotBER
from sionna.phy.ofdm import KBestDetector, LinearDetector
from sionna.phy.mimo import StreamManagement

# Import Sionna RT components
from sionna.rt import load_scene, Camera, Transmitter, Receiver, PlanarArray,\
                      PathSolver, RadioMapSolver

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

# Setting up the Ray Tracer

Let’s start by defining some constants that control the system we want to simulate.

In [2]:
# System parameters
subcarrier_spacing = 30e3 # Hz
num_time_steps = 14 # Total number of ofdm symbols per slot
carrier_frequency = 3.5e9 # Hz

# num_tx = 4 # Number of users
# num_rx = 1 # Only one receiver considered
# num_tx_ant = 4 # Each user has 4 antennas
# num_rx_ant = 16 # The receiver is equipped with 16 antennas

num_tx = 2 # Number of users
num_rx = 1 # Only one receiver considered
num_tx_ant = 1 # Each user has 4 antennas
num_rx_ant = 2 # The receiver is equipped with 16 antennas

# batch_size for CIR generation
# batch_size_cir = 1000
batch_size_cir = 6

We then set up the radio propagation environment. We start by loading a scene and then add a transmitter that acts as a base station. We will later use channel reciprocity to simulate the uplink direction.

In [12]:
# Load an integrated scene.
# You can try other scenes, such as `sionna.rt.scene.etoile`. Note that this would require
# updating the position of the transmitter (see below in this cell).
# scene = load_scene(sionna.rt.scene.munich)
scene = load_scene("test2.xml")
# scene.preview()

# Transmitter (=basestation) has an antenna pattern from 3GPP 38.901
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=num_rx_ant//2, # We want to transmitter to be equiped with 2 antennas
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="cross")

# Create transmitter
tx1 = Transmitter(name="tx1",
                 position=[30,46,40],
                 look_at=[30,0,10], # optional, defines view direction
                 power_dbm=23,
                 display_radius=3.) # optinal, radius of the sphere for visualizing the device

tx2 = Transmitter(name="tx2",
                 position=[30,46,22],
                 look_at=[30,0,10], # optional, defines view direction
                 power_dbm=23,
                 display_radius=3.) # optinal, radius of the sphere for visualizing the device

# scene.preview()
scene.add(tx1)
scene.add(tx2)

# Create new camera
bird_cam = Camera(position=[0,80,500], orientation=np.array([0,np.pi/2,-np.pi/2]))

In [13]:
# --- 放在 tx1 = Transmitter(...) 後面 ---
import json
import numpy as np
import pymap3d as pm

# 將各種數值型別（Tensor/ndarray/drjit 等）安全轉成 Python float
def to_float(v):
    if hasattr(v, "numpy"):          # TensorFlow / PyTorch 等
        v = v.numpy()
    if hasattr(v, "__array__"):
        v = np.asarray(v)
    if hasattr(v, "item"):
        v = v.item()
    return float(v)

# 1) 你的 OSM 外框（度）
min_lat = 24.7831
max_lat = 24.7909
min_lon = 120.9935
max_lon = 121.0024

# 2) 以外框中心當原點（WGS-84）；h0=原點高度(公尺)，沒有地形就先 0
lat0 = (min_lat + max_lat) / 2.0
lon0 = (min_lon + max_lon) / 2.0
h0   = 0.0

# 3) ENU(公尺) -> 經緯高
x, y, z = (to_float(c) for c in tx1.position)     # e, n, u（公尺）
lat, lon, h = pm.enu2geodetic(x, y, z, lat0, lon0, h0)
lat, lon, h = to_float(lat), to_float(lon), to_float(h)

# 4) 組 JSON 並輸出
out = {
    "bbox": {
        "min_lat": round(min_lat, 7),
        "max_lat": round(max_lat, 7),
        "min_lon": round(min_lon, 7),
        "max_lon": round(max_lon, 7)
    },
    "origin": {"lat": round(lat0, 7), "lon": round(lon0, 7), "h": float(h0)},
    "transmitter": {
        "name": getattr(tx1, "name", "tx"),
        "position_xyz_m": [float(x), float(y), float(z)],
        "geodetic": {"lat": round(lat, 7), "lon": round(lon, 7), "h": round(h, 3)}
    }
}

with open("uav_from_sionna.json", "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)
print("[export] Wrote uav_from_sionna.json")


[export] Wrote uav_from_sionna.json


We then compute a radio map for the instantiated transmitter.

In [5]:
# max_depth = 5
max_depth = 12

# Radio map solver
rm_solver = RadioMapSolver()

# Compute the radio map
rm = rm_solver(scene,
               max_depth=12,
               cell_size=(1., 1.),
               samples_per_tx=10**7)

Let’s visualize the computed radio map.

In [6]:
if no_preview:
    # Render an image
    scene.render(camera=bird_cam,
                 radio_map=rm,
                 rm_vmin=-110,
                 clip_at=12.); # Clip the scene at rendering for visualizing the refracted field
else:
    # Show preview
    scene.preview(radio_map=rm,
                  rm_vmin=-110,
                  clip_at=40.); # Clip the scene at rendering for visualizing the refracted field

The function RadioMap.sample_positions() allows sampling of random user positions from a radio map. It ensures that only positions that have a path gain of at least min_gain_dB dB and at most max_gain_dB dB are sampled, i.e., it ignores positions without a connection to the transmitter. Further, one can set the distances min_dist and max_dist to sample only points within a certain distance range from the transmitter.

In [7]:
min_gain_db = -130 # in dB; ignore any position with less than -130 dB path gain
max_gain_db = 0 # in dB; ignore strong paths

# Sample points in a 5-400m range around the receiver
min_dist = 5 # in m
max_dist = 400 # in m

# Sample batch_size random user positions from the radio map
# ue_pos, _ = rm.sample_positions(num_pos=batch_size_cir,
#                                 metric="path_gain",
#                                 min_val_db=min_gain_db,
#                                 max_val_db=max_gain_db,
#                                 min_dist=min_dist,
#                                 max_dist=max_dist)

ue_pos = [[20,0,1.5], [30,0,1.5], [20,10,1.5], [30,10,1.5], [20,-10,1.5], [30,-10,1.5]]

We now add new receivers (=UEs) at the sampled positions.

Remark: This is an example for 5G NR PUSCH (uplink direction), we will reverse the direction of the channel for later BER simulations.

In [8]:
# Configure antenna array for all receivers (=UEs)
# scene.rx_array = PlanarArray(num_rows=1,
#                              num_cols=num_tx_ant//2, # Each receiver is equipped with 4 antennas
#                              vertical_spacing=0.5,
#                              horizontal_spacing=0.5,
#                              pattern="iso",
#                              polarization="cross")

scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=num_tx_ant, # Each receiver is equipped with 1 antennas
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")
print(type(ue_pos))
print(len(ue_pos))       # 最外層長度
print(len(ue_pos[0]))    # ue_pos[0] 長度
print(ue_pos[:5])        # 看前五個值

# Create batch_size receivers
for i in range(batch_size_cir):
    scene.remove(f"rx-{i}") # Remove old receiver if any
    rx = Receiver(name=f"rx-{i}",
                  position=ue_pos[i], # Position sampled from radio map
                  velocity=(3.,3.,0),
                  display_radius=3., # optional, radius of the sphere for visualizing the device
                  color=(0,1,0) # optional, color for visualizing the device
                  )
    scene.add(rx)
p_solver = PathSolver()
paths = p_solver(scene, max_depth=5)

if no_preview:
    # Render an image
    scene.render(camera=bird_cam,
                 radio_map=rm,
                 rm_vmin=-110,
                 clip_at=12.); # Clip the scene at rendering for visualizing the refracted field
else:
    # Show preview
    scene.preview(radio_map=rm,
                  rm_vmin=-110,
                  clip_at=40.); # Clip the scene at rendering for visualizing the refracted field

<class 'list'>
6
3
[[20, 0, 1.5], [30, 0, 1.5], [20, 10, 1.5], [30, 10, 1.5], [20, -10, 1.5]]


Each dot represents a receiver position drawn from the random sampling function of the radio map. This allows to efficiently sample batches of random channel realizations even in complex scenarios.

# Creating a CIR Dataset

We can now simulate the CIRs for many different positions which will be used later on as a dataset for link-level simulations.

Remark: Running the cells below can take some time depending on the requested number of CIRs.

In [9]:
# target_num_cirs = 5000 # Defines how many different CIRs are generated.
target_num_cirs = 2 # Defines how many different CIRs are generated.
# Remark: some path are removed if no path was found for this position

max_depth = 5
min_gain_db = -130 # in dB / ignore any position with less than -130 dB path gain
max_gain_db = 0 # in dB / ignore any position with more than 0 dB path gain

# Sample points within a 10-400m radius around the transmitter
min_dist = 10 # in m
max_dist = 400 # in m

# List of channel impulse reponses
a_list = []
tau_list = []

# Maximum number of paths over all batches of CIRs.
# This is used later to concatenate all CIRs.
max_num_paths = 0

# Path solver
p_solver = PathSolver()

# Each simulation returns batch_size_cir results
num_runs = int(np.ceil(target_num_cirs/batch_size_cir))
for idx in range(num_runs):
    print(f"Progress: {idx+1}/{num_runs}", end="\r")

    # Sample random user positions
    # ue_pos, _ = rm.sample_positions(
    #                     num_pos=batch_size_cir,
    #                     metric="path_gain",
    #                     min_val_db=min_gain_db,
    #                     max_val_db=max_gain_db,
    #                     min_dist=min_dist,
    #                     max_dist=max_dist,
    #                     seed=idx) # Change the seed from one run to the next to avoid sampling the same positions

    ue_pos = [[[30,0,1.5], [20,0,1.5]]]
    
    # Update all receiver positions
    for rx in range(batch_size_cir):
        scene.receivers[f"rx-{rx}"].position = ue_pos[rx]

    # Simulate CIR
    paths = p_solver(scene, max_depth=max_depth, max_num_paths_per_src=10**7)

    # Transform paths into channel impulse responses
    a, tau = paths.cir(sampling_frequency=subcarrier_spacing,
                         num_time_steps=14,
                         out_type='numpy')
    
    a_list.append(a)
    tau_list.append(tau)

    # Update maximum number of paths over all batches of CIRs
    num_paths = a.shape[-2]
    if num_paths > max_num_paths:
        max_num_paths = num_paths

# Show path
if no_preview:
    # Render an image
    scene.render(camera=bird_cam,
                 radio_map=rm,
                 rm_vmin=-110,
                 paths=paths,
                 clip_at=12.); # Clip the scene at rendering for visualizing the refracted field
else:
    # Show preview
    scene.preview(radio_map=rm,
                  rm_vmin=-110,
                  paths=paths,
                  clip_at=40.); # Clip the scene at rendering for visualizing the refracted field


# Concatenate all the CIRs into a single tensor along the num_rx dimension.
# First, we need to pad the CIRs to ensure they all have the same number of paths.
a = []
tau = []
for a_,tau_ in zip(a_list, tau_list):
    num_paths = a_.shape[-2]
    a.append(np.pad(a_, [[0,0],[0,0],[0,0],[0,0],[0,max_num_paths-num_paths],[0,0]], constant_values=0))
    tau.append(np.pad(tau_, [[0,0],[0,0],[0,max_num_paths-num_paths]], constant_values=0))
a = np.concatenate(a, axis=0) # Concatenate along the num_rx dimension
tau = np.concatenate(tau, axis=0)

# Let's now convert to uplink direction, by switing the receiver and transmitter
# dimensions
a = np.transpose(a, (2,3,0,1,4,5))
tau = np.transpose(tau, (1,0,2))

# Add a batch_size dimension
a = np.expand_dims(a, axis=0)
tau = np.expand_dims(tau, axis=0)

# Exchange the num_tx and batchsize dimensions
a = np.transpose(a, [3, 1, 2, 0, 4, 5, 6])
tau = np.transpose(tau, [2, 1, 0, 3])

# Remove CIRs that have no active link (i.e., a is all-zero)
p_link = np.sum(np.abs(a)**2, axis=(1,2,3,4,5,6))
a = a[p_link>0.,...]
tau = tau[p_link>0.,...]

print("Shape of a:", a.shape)
print("Shape of tau: ", tau.shape)

np.savez("cir_nycu_2RU_2UE.npz", a=a, tau=tau)

TypeError: mitsuba.Point3f.__init__(): Input has the wrong size (expected 3 elements, got 2).

Note that transmitters and receivers have been reversed, i.e., the transmitter now denotes the UE (with 4 antennas each) and the receiver is the base station (with 16 antennas).

Remark: We have removed all positions for which the resulting CIR had zero gain, i.e., there was no path between the transmitter and the receiver. This comes from the fact that the RadioMap.sample_positions() function samples from a radio map subdivided into cells and randomizes the position within the cells. Therefore, randomly sampled positions may have no paths connecting them to the transmitter.

Let us now define a data generator that samples random UEs from the dataset and yields the previously simulated CIRs.

In [ ]:
class CIRGenerator:
    """Creates a generator from a given dataset of channel impulse responses

    The generator samples ``num_tx`` different transmitters from the given path
    coefficients `a` and path delays `tau` and stacks the CIRs into a single tensor.

    Note that the generator internally samples ``num_tx`` random transmitters
    from the dataset. For this, the inputs ``a`` and ``tau`` must be given for
    a single transmitter (i.e., ``num_tx`` =1) which will then be stacked
    internally.

    Parameters
    ----------
    a : [batch size, num_rx, num_rx_ant, 1, num_tx_ant, num_paths, num_time_steps], complex
        Path coefficients per transmitter

    tau : [batch size, num_rx, 1, num_paths], float
        Path delays [s] per transmitter

    num_tx : int
        Number of transmitters

    Output
    -------
    a : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps], tf.complex
        Path coefficients

    tau : [batch size, num_rx, num_tx, num_paths], tf.float
        Path delays [s]
    """

    def __init__(self,
                 a,
                 tau,
                 num_tx):

        # Copy to tensorflow
        self._a = tf.constant(a, tf.complex64)
        self._tau = tf.constant(tau, tf.float32)
        self._dataset_size = self._a.shape[0]

        self._num_tx = num_tx

    def __call__(self):

        # Generator implements an infinite loop that yields new random samples
        while True:
            # Sample 4 random users and stack them together
            idx,_,_ = tf.random.uniform_candidate_sampler(
                            tf.expand_dims(tf.range(self._dataset_size, dtype=tf.int64), axis=0),
                            num_true=self._dataset_size,
                            num_sampled=self._num_tx,
                            unique=True,
                            range_max=self._dataset_size)

            a = tf.gather(self._a, idx)
            tau = tf.gather(self._tau, idx)

            # Transpose to remove batch dimension
            a = tf.transpose(a, (3,1,2,0,4,5,6))
            tau = tf.transpose(tau, (2,1,0,3))

            # And remove batch-dimension
            a = tf.squeeze(a, axis=0)
            tau = tf.squeeze(tau, axis=0)

            yield a, tau

We use Sionna’s built-in CIRDataset to initialize a channel model that can be directly used in Sionna’s OFDMChannel layer.

In [ ]:
batch_size = 20 # Must be the same for the BER simulations as CIRDataset returns fixed batch_size

# Init CIR generator
cir_generator = CIRGenerator(a,
                             tau,
                             num_tx)
# Initialises a channel model that can be directly used by OFDMChannel layer
channel_model = CIRDataset(cir_generator,
                           batch_size,
                           num_rx,
                           num_rx_ant,
                           num_tx,
                           num_tx_ant,
                           max_num_paths,
                           num_time_steps)